**Conecting to Drive**

In [ ]:
from google.colab import drive

import numpy as np
import pandas as pd
import tensorflow as tf
import scipy.io as sio

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Modules

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

def load_data(data, window_size=1, scaler=None, scale='linear', logmin=-55, shuffle=True, train_size=2900, 
              data_format='channels_last', seed=0):
    """
    Parameters
    -----------
    data : ndarray
      Simulation data, `shape=(steps_size+1, features, num_simuls)`
    window_size : integer, optional
      LSTM timesteps
    scaler : `Scaler`, optional
      `None` (default), or `Scaler` to normalization
    shuffle : boolean, optional
      Shuffle dataset
    train_size : integer
      Number of simulation used for training
    data_format : string, optional
      `'channels_last'` (default), or `'channels_first'`
    seed : integer, optional
      Seed for reproducibility

    Returns
    -----------
    x_train : ndarray
      Training data input, `shape=(train_size*steps_size, window_size, features)`
    x_test : ndarray
      Test data input, `shape=(test_size*steps_size, window_size, features)`
    y_train : ndarray
      Training data output, `shape=(train_size*steps_size, features)`
    y_test : ndarray
      Test data output, `shape=(test_size*steps_size, features)`
    scaler : `Scaler`
      `Scaler` to denormalization
    """
    # for reproducibility
    np.random.seed(seed=seed)

    # steps, features, num_simulations
    N, F, S = data.shape
    N = N - 1                             # steps size
    ts = window_size + 1                  # window size (timesteps)

    # transform data
    if scale == 'log':
        data = 10*np.log10(data)
        data[data<logmin] = logmin

    # normalize data
    if scaler == None:
        scaler = MinMaxScaler()
        norm = scaler.fit_transform(data.reshape([-1,1]))
    else:
        norm = scaler.transform(data.reshape([-1,1]))

    data = norm.reshape(data.shape)

    # tile the beginning of the evolution with 'window_size' input profiles
    cold = np.tile(data[0,:,:], (window_size-1,1,1))
    data = np.concatenate((cold,data))

    # dataset
    dataset = np.zeros((S*N,ts,F))

    range_S = np.arange(S)
    if shuffle:
        np.random.shuffle(range_S)

    s = 0
    for m in range_S:
        for n in range(N):
            dataset[s] = data[n:n+ts,:,m]
            s += 1

    # dataset division:
    x_data = dataset[:,0:ts-1,:]
    y_data = dataset[:,ts-1,:]

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=train_size*N, shuffle=False)

    return x_train, x_test, y_train, y_test, scaler

def build_model(input, output, lstm_units=300, lstm_actv='tanh', 
                dense_units=161, dense_actv='relu', dropout_rate=0.0, 
                optimizer='Adam', learning_rate=1e-4, loss='mse'):
    """
    Parameters
    -----------
    input : ndarray
      Training data input, `shape=(train_size*steps_size, window_size, features)`
    output : ndarray
      Training data output, `shape=(train_size*steps_size, features)`
    lstm_units : integer, optional
      Units in LSTM layer
    lstm_actv : string, optional
      Activation function of the LSTM layer
    dense_units : integer, optional
      Units in Dense layer
    dense_actv : string, optional
      Activation function of the Dense layer
    optimizer : string, optional
      Optimizer method: `RMSProp`, `Adam` (Default), or `AdamW`
    dropout_rate : float, optional
      Float between 0 and 1. Fraction of the input units to drop.
    loss : string, optional
      Optimizer loss function.

    Returns
    -----------
    model : keras model
    """
    # define model architecture
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.LSTM(lstm_units, activation=lstm_actv, return_sequences=False, input_shape=input.shape[1:]))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(dense_units, activation=dense_actv))
    model.add(tf.keras.layers.Dense(dense_units, activation=dense_actv))
    model.add(tf.keras.layers.Dense(output.shape[1], activation='sigmoid'))

    # compile model
    if optimizer == 'RMSProp':
        opt = tf.keras.optimizers.RMSprop(learning_rate, rho=0.9)
    elif optimizer == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate, epsilon=1e-7)

    model.compile(loss=loss, optimizer=opt)
    return model

def update_model(model, optimizer='Adam', learning_rate=1e-5, loss='mse'):
    """
    Parameters
    -----------
    model : keras model
      Keras model to update
    optimizer : string, optional
      Optimizer method: `RMSProp`, `Adam` (Default)
    learning_rate : float, optional
      Optimizer learning rate
    loss : string, optional
      Optimizer loss function

    Returns
    -----------
    model : keras model
      Updated keras model
    """
    # compile model
    if optimizer == 'RMSProp':
        opt = tf.keras.optimizers.RMSprop(learning_rate, rho=0.9)
    elif optimizer == 'Adam':
        opt = tf.keras.optimizers.Adam(learning_rate, epsilon=1e-7)

    model.compile(loss=loss, optimizer=opt)
    return model

def predict(model, x_test, steps_size):
    """
    Parameters
    -----------
    model : keras model
    x_test : ndarray
      Test data input, `shape=(test_size*steps_size, window_size, features)`
    steps_size : integer
      Number of propagation steps

    Returns
    -----------
    y_pred : ndarray
      Estimated data output, `shape=(test_size*steps_size, features)`
    """
    N = steps_size
    batch, timesteps, features = x_test.shape

    # inputs
    x_pred = x_test[0::N]
    y_pred = np.zeros((batch, 1, features))

    # predict
    for n in range(N):
        y_pred[n::N,0,:] = model.predict(x_pred, verbose=0)
        x_pred = np.concatenate((x_pred[:,1:,:], y_pred[n::N]), axis=1)

    return y_pred.squeeze()

# Propagation Problem

## Data

In [ ]:
# load data with scipy.io:
path = '/content/drive/MyDrive/Colab Notebooks/Mestrado/Design and analysis of recurrent neural networks for ultrafast optical pulses nonlinear propagation (2022)'

# params
window_size = 10      # window size (timesteps)
N = 101 - 1           # steps size

data = sio.loadmat(datapath + '/data/sech_time.mat')['data']
# data = sio.loadmat(path + '/data/gaussian_time.mat')['data']
# data = sio.loadmat(path + '/data/sech_P40W_T3ps_time.mat')['data']
# data = sio.loadmat(path + '/data/sech_P50W_T5ps_time.mat')['data']
# data = sio.loadmat(path + '/data/sech_P70W_T5ps_time.mat')['data']

x_train, x_test, y_train, y_test, scaler = load_data(data, window_size, scale='linear')

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(290000, 10, 256) (10000, 10, 256) (290000, 256) (10000, 256)


## Model

In [ ]:
# model
lstm_units, dense_units = 512, 161

model = build_model(x_train, y_train, lstm_units=lstm_units, lstm_actv='tanh', 
                    dense_units=dense_units, dropout_rate=0.0, optimizer='RMSProp')

model.summary()

checkpointer = tf.keras.callbacks.ModelCheckpoint(path+ '/nets/LSTM.h5', 
                                                  verbose=0, save_best_only=True)

history = model.fit(x_train, y_train, epochs=30, validation_split=0.1, 
                    shuffle=True, verbose=2, callbacks=[checkpointer])

#################################### UPDATE ####################################
model = update_model(model, optimizer='RMSProp', learning_rate=1e-5)

history = model.fit(x_train, y_train, epochs=30, validation_split=0.1, 
                    shuffle=True, verbose=2, callbacks=[checkpointer])

# #################################### UPDATE ####################################
model = update_model(model, optimizer='RMSProp', learning_rate=1e-6)

history = model.fit(x_train, y_train, epochs=30, validation_split=0.1, 
                    shuffle=True, verbose=2, callbacks=[checkpointer])

model.load_weights(path + '/nets/LSTM.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 256)           525312    
                                                                 
 lstm_1 (LSTM)               (None, 10, 256)           525312    
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 256)               6

Different values can be found given the stochastic nature of the model.

## Metrics

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# Step-by-Step prediction:
y_pred = predict(model, x_test, N)

# Metrics
MSE = mean_squared_error(y_test, y_pred, squared=True)
RMSE = mean_squared_error(y_test, y_pred, squared=False)
NRMSE = np.sqrt(np.sum((y_test - y_pred)**2)/np.sum(y_test**2))
R2 = r2_score(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)

print('RMSE = %.6f' % RMSE)
print('NRMSE = %.6f' % NRMSE)
print('MSE  = %.6f' % MSE)
print('MAE  = %.6f' % MAE)
print('R2   = %.6f' % R2)

RMSE = 0.024404
NRMSE = 0.045293
MSE  = 0.000887
MAE  = 0.013804
R2   = 0.866934


The best networks weights are saved

In [ ]:
data = {'Nets': 'LSTM'+str(lstm_units)+'_2Dense'+str(dense_units)+'_ts'+str(window_size)+'_sech_time',
        'ts': window_size,
        'RMSE': RMSE,
        'NRMSE': NRMSE,
        'MSE': MSE, 
        'MAE': MAE, 
        'R2': R2,
        'Details': '''def build_model(input, output, lstm_units=300, lstm_actv='tanh', 
                                      dense_units=161, dense_actv='relu', dropout_rate=0.0, 
                                      loss='mean_squared_error'):
  
                        # define model architecture
                        model = tf.keras.models.Sequential()

                        model.add(tf.keras.layers.LSTM(lstm_units, activation=lstm_actv, return_sequences=False, input_shape=input.shape[1:]))
                        model.add(tf.keras.layers.Dropout(dropout_rate))
                        model.add(tf.keras.layers.Dense(dense_units, activation=dense_actv))
                        model.add(tf.keras.layers.Dense(dense_units, activation=dense_actv))
                        model.add(tf.keras.layers.Dense(output.shape[1], activation='sigmoid'))

                        return model'''}

dataframe = pd.DataFrame(data=data, index=[0])

dataframe.to_csv(path + '/results/LSTM'+str(lstm_units)+'_2Dense'+str(dense_units)+'_ts'+str(window_size)+'_sech_time.csv', header=True)
model.save(path + '/nets/LSTM'+str(lstm_units)+'_2Dense'+str(dense_units)+'_ts'+str(window_size)+'_sech_time.h5')